In [21]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [22]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from wandb.integration.keras import WandbMetricsLogger

import wandb
import params

In [23]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="training")

In [24]:
processed_data_at = run.use_artifact(f'{params.PROCESSED_DATA_AT}:latest')



In [25]:
%ls artifacts/flowers_simples_split:v3/flowers


daisy/  dandelion/  rose/  sunflower/  tulip/


In [26]:
df = pd.read_csv("data_split.csv")
df.head()

,fname,tulip,dandelion,daisy,rose,sunflower
0,8723767533_9145dec4bd_n.jpg,1,0,0,0,0
1,5716293002_a8be6a6dd3_n.jpg,1,0,0,0,0
2,8521597402_4b6169ba05.jpg,1,0,0,0,0
3,8713388322_e5ae26263b_n.jpg,1,0,0,0,0
4,4624404489_11e10fcd33_n.jpg,1,0,0,0,0


In [ ]:
dataset_dir = 'artifacts/flowers_simples_split:v3/flowers'

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Gerador para treinamento
train_generator = datagen.flow_from_directory(
    directory=dataset_dir,
    target_size=(180, 180),
    batch_size=32,
    class_mode='categorical',
    subset='training' 
)

# Gerador para validação
validation_generator = datagen.flow_from_directory(
    directory=dataset_dir,
    target_size=(180, 180),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 3457 images belonging to 5 classes.
Found 860 images belonging to 5 classes.


In [28]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(180, 180, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(5, activation='softmax') 
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


/home/gabs/miniconda3/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-12-13 00:33:58.404406: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 178, 178, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 89, 89, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 87, 87, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 43, 43, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 41, 41, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 51200)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │     1,638,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,731,845 (6.61 MB)

 Trainable params: 1,731,845 (6.61 MB)

 Non-trainable params: 0 (0.00 B)

In [29]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[wandb.keras.WandbMetricsLogger(),
                     wandb.keras.WandbModelCheckpoint(filepath='model.keras', save_best_only=True)])

wandb: WARNING When using `save_best_only`, ensure that the `filepath` argument contains formatting placeholders like `{epoch:02d}` or `{batch:02d}`. This ensures correct interpretation of the logged artifacts.
/home/gabs/miniconda3/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


2024-12-13 00:33:59.805115: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 129777664 exceeds 10% of free system memory.
2024-12-13 00:33:59.881623: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 62005248 exceeds 10% of free system memory.
2024-12-13 00:34:00.107536: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 62005248 exceeds 10% of free system memory.
2024-12-13 00:34:00.408354: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 64888832 exceeds 10% of free system memory.
2024-12-13 00:34:00.408424: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 129777664 exceeds 10% of free system memory.


109/109 ━━━━━━━━━━━━━━━━━━━━ 85s 769ms/step - accuracy: 0.3359 - loss: 1.4948 - val_accuracy: 0.5477 - val_loss: 1.1454
Epoch 2/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 82s 755ms/step - accuracy: 0.5808 - loss: 1.0602 - val_accuracy: 0.5965 - val_loss: 1.0524
Epoch 3/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 80s 725ms/step - accuracy: 0.6474 - loss: 0.8774 - val_accuracy: 0.6012 - val_loss: 1.0602
Epoch 4/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 80s 729ms/step - accuracy: 0.6884 - loss: 0.8291 - val_accuracy: 0.6163 - val_loss: 0.9857
Epoch 5/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 77s 703ms/step - accuracy: 0.7500 - loss: 0.6341 - val_accuracy: 0.6477 - val_loss: 1.1188
Epoch 6/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 76s 696ms/step - accuracy: 0.8284 - loss: 0.4684 - val_accuracy: 0.6244 - val_loss: 1.2363
Epoch 7/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 78s 717ms/step - accuracy: 0.8770 - loss: 0.3298 - val_accuracy: 0.6500 - val_loss: 1.2680
Epoch 8/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 76s 694ms/step - accuracy: 0.9307 - loss: 0.2149 - val

In [30]:
wandb.finish()

epoch/accuracy,▁▃▄▄▅▆▇▇██
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▇▆▅▄▃▂▂▁▁
epoch/val_accuracy,▁▄▅▆█▆█▇▇▆
epoch/val_loss,▂▂▂▁▂▃▃▆▆█
epoch/accuracy,0.97165
epoch/epoch,9
epoch/learning_rate,0.001
epoch/loss,0.10162
epoch/val_accuracy,0.61512
